In [ ]:
from data_access import events_dataframe, update_data_file, DATA_FILE
from plots import cumulative_history_plot, add_interpoop_stats

In [ ]:
%load_ext autoreload


In [ ]:
%autoreload 2

In [ ]:
df.info()

In [ ]:
df = events_dataframe()
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta

def plot_feeding_data(df, tz='US/Pacific', oz_per_15_minutes_boobs=None):
    for col in ['amount_oz', 'time_left', 'time_right']:
        df[col] = pd.to_numeric(df['amount_oz'])
    
    if oz_per_15_minutes_boobs is not None:
        df.loc[df['event_type'] == 'breastfeeding', 'amount_oz'] = \
            df[df['event_type'] == 'breastfeeding'].apply(
                lambda x: (x['time_left'] + x['time_right']) / 15 * oz_per_15_minutes_boobs,
                axis=1
            )
    # Step 1: Filter to include only 'feeding' and 'breastfeeding' event types
    df = df[df['event_type'].isin(['feeding', 'breastfeeding'])]
    df = df.sort_values(by='timestamp', ascending=True)
    
    # Step 2: Convert timestamp to datetime and extract date
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s', utc=True).dt.tz_convert(tz)
    df['date'] = df['timestamp'].dt.date
    df['hour_of_day'] = df['timestamp'].dt.hour + df['timestamp'].dt.minute / 60
    
    # Step 3: Filter to include only the most recent 7 days
    end_date = df['date'].max()
    start_date = end_date - timedelta(days=6)
    df_recent = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
    
    # Step 4: Calculate cumulative amount_oz for each day
    df['amount_oz'] = pd.to_numeric(df['amount_oz'], errors='coerce')
    df = df.dropna(subset=['amount_oz'])
    
    df['cumulative_amount_oz'] = df.groupby('date')['amount_oz'].cumsum()
    
    # Step 5: Calculate averages and standard deviations
    start_date_28d = end_date - timedelta(days=27)
    df_28d = df[(df['date'] >= start_date_28d) & (df['date'] <= end_date)]
    
    avg_7d = df_recent.groupby('hour_of_day')['amount_oz'].mean().cumsum()
    std_7d = df_recent.groupby('hour_of_day')['amount_oz'].std().cumsum()
    
    avg_28d = df_28d.groupby('hour_of_day')['amount_oz'].mean().cumsum()
    std_28d = df_28d.groupby('hour_of_day')['amount_oz'].std().cumsum()
    
    # Step 6: Plot the data using Plotly
    fig = go.Figure()
    colors = px.colors.qualitative.Plotly
    event_type_to_marker = {
        'feeding': 'circle',
        'breastfeeding': 'star'
    }
    df_recent['marker_symbol'] = df_recent['event_type'].map(event_type_to_marker)
    
    for i, (date, group) in enumerate(df_recent.groupby('date')):
        group = pd.concat([pd.DataFrame(
            {
                "hour_of_day": [0],
                "cumulative_amount_oz": [0],
                "marker_symbol": 'circle',
            }
        ), group], ignore_index=True)
        fig.add_trace(go.Scatter(
            x=group['hour_of_day'],
            y=group['cumulative_amount_oz'],
            mode='lines+markers',
            name=str(date),
            line=dict(color=colors[i % len(colors)]),
            marker=dict(symbol=group['marker_symbol'], color=colors[i % len(colors)]),
        ))
    
    # Add average and standard deviation lines for the past 7 days
    # fig.add_trace(go.Scatter(
    #     x=avg_7d.index,
    #     y=avg_7d,
    #     mode='lines',
    #     name='7D Avg',
    #     line=dict(color='black', dash='dash')
    # ))
    # fig.add_trace(go.Scatter(
    #     x=avg_7d.index,
    #     y=avg_7d + std_7d,
    #     mode='lines',
    #     name='7D Avg + 1 Std',
    #     line=dict(color='gray', dash='dot')
    # ))
    # fig.add_trace(go.Scatter(
    #     x=avg_7d.index,
    #     y=avg_7d - std_7d,
    #     mode='lines',
    #     name='7D Avg - 1 Std',
    #     line=dict(color='gray', dash='dot')
    # ))
    
    # Add average and standard deviation lines for the past 28 days
#     fig.add_trace(go.Scatter(
#         x=avg_28d.index,
#         y=avg_28d,
#         mode='lines',
#         name='28D Avg',
#         line=dict(color='blue', dash='dash')
#     ))
#     fig.add_trace(go.Scatter(
#         x=avg_28d.index,
#         y=avg_28d + std_28d,
#         mode='lines',
#         name='28D Avg + 1 Std',
#         line=dict(color='lightblue', dash='dot')
#     ))
#     fig.add_trace(go.Scatter(
#         x=avg_28d.index,
#         y=avg_28d - std_28d,
#         mode='lines',
#         name='28D Avg - 1 Std',
#         line=dict(color='lightblue', dash='dot')
#     ))
    
    fig.update_layout(
        title='Cumulative Feeding Amount Over Time',
        xaxis_title=f'Hour of Day ({tz})',
        yaxis_title='Cumulative Amount Eaten (oz)',
        xaxis=dict(tickformat='%H:%M'),
        legend_title='Date'
    )
    
    return fig

# Example usage:
# Assuming df is your DataFrame
# plot_feeding_data(df)


# Example usage:
# Assuming df is your DataFrame
plot_feeding_data(df, oz_per_15_minutes_boobs=1)



In [ ]:
import pandas as pd
import plotly.graph_objects as go

In [ ]:
!pwd